Mounting Gdrive & ...

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount GDrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/ML_Data G1.rar" .

In [ ]:
 !unrar x "ML_Data G1.rar"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from ML_Data G1.rar

Creating    ML_Data G1                                                OK
Creating    ML_Data G1/Bandari                                        OK
Extracting  ML_Data G1/Bandari/1.mp3                                       0%  OK 
Extracting  ML_Data G1/Bandari/10.mp3                                      0%  OK 
Extracting  ML_Data G1/Bandari/100.mp3                                     0%  OK 
Extracting  ML_Data G1/Bandari/101.mp3                                     0%  OK 
Extracting  ML_Data G1/Bandari/102.mp3                                     0%  OK 
Extracting  ML_Data G1/Bandari/103.mp3                                     0%  OK 
Extracting  ML_Data G1/Bandari/104.mp3                                     0%  OK 
Extracting  ML_Data G1/Bandari/105.mp3                                     0%  OK 
Extracting  ML_Da

Importing libraries

In [8]:
import numpy as np
import csv
import os
import warnings
import pandas as pd
import librosa
import librosa.display

In [2]:
warnings.filterwarnings('ignore')

In [5]:
header = 'filename chroma_stft rms spectral_centroid spectral_bandwidth rolloff zero_crossing_rate tempo'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' class'
header = header.split()

In [ ]:
file = open('data_Final.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
classes = 'Bandari Gilaki Kordi Lori Torki'.split()
for c in classes:
    for filename in os.listdir(f'/content/ML_Data G1/{c}'):
        songname = f'/content/ML_Data G1/{c}/{filename}'
        # apparently some files in dataset were corrupted so we had to catch exceptions that occurred while extracting features from audio files
        try: 
            # Extracting features such as  temp, zero crossing rate,spectral centroid & etc.
            y, sr = librosa.load(songname)
            tempo = librosa.beat.tempo(y=y, sr=sr)
            zcr = librosa.feature.zero_crossing_rate(y)
            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            rmse = librosa.feature.rms(y=y)
            spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            mfcc = librosa.feature.mfcc(y=y, sr=sr)
            to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)} {np.mean(tempo)}'    
            for e in mfcc:
                to_append += f' {np.mean(e)}'
            to_append += f' {c}'
            file = open('data_Final.csv', 'a', newline='')
            with file:
                writer = csv.writer(file)
                writer.writerow(to_append.split())
        except:
            print("An exception occurred while reading %s" %(songname))

An exception occurred while reading /content/ML_Data G1/Bandari/22.mp3
An exception occurred while reading /content/ML_Data G1/Bandari/28.mp3


Generated Dataset:

In [7]:
data = pd.read_csv('data_Final.csv')
data = data.sample(frac=1).reset_index(drop=True)
data.describe()

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,tempo,mfcc1,mfcc2,mfcc3,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
count,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,...,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000
mean,0.329752,0.192873,2387.294201,2364.037422,4966.152649,0.116026,122.603569,-97.384672,87.303162,-7.582925,...,-4.583102,0.125290,-3.391558,0.077267,-4.601739,0.197782,-6.148117,0.069985,-4.170508,-0.740065
std,0.061263,0.076878,572.025510,446.092498,1258.716485,0.040212,18.327290,72.257997,29.651303,23.541774,...,5.525813,5.081117,5.144109,4.637577,4.295119,4.238548,3.976410,3.975177,3.742726,4.069368
min,0.176051,0.006707,189.221640,726.967972,223.990196,0.000298,66.256010,-816.888916,-22.683443,-117.896515,...,-33.037895,-22.438185,-28.786459,-17.809240,-25.124331,-19.695129,-21.542820,-15.078506,-18.323135,-21.080341
25%,0.286423,0.135361,2033.364143,2148.762919,4128.209468,0.089982,112.347147,-127.964371,70.409042,-16.979990,...,-7.649524,-3.120913,-6.033784,-2.620801,-7.090947,-2.195034,-8.497351,-2.269188,-6.488165,-3.216138
50%,0.327524,0.196164,2391.470056,2476.646875,5125.766007,0.109761,123.046875,-83.398636,82.845978,-2.212324,...,-4.214223,0.310377,-2.953628,0.387445,-4.513611,0.457979,-6.119240,0.312792,-4.203309,-0.483301
75%,0.368728,0.246867,2747.208888,2681.314348,5900.965125,0.133689,135.999178,-50.656313,100.000504,7.381118,...,-1.104499,3.471175,-0.270714,3.137032,-2.024628,2.829406,-3.737077,2.643852,-2.136729,1.750029
max,0.906352,0.558828,4853.764277,3365.185920,8756.623719,0.357932,184.570312,62.921417,233.426758,104.796150,...,23.110296,18.386873,24.856686,19.551630,24.232126,23.810650,14.026166,11.563221,14.946872,17.250811


##### The very first step in our project was to extract features and components from the audio files and since we didn't have enough experience dealing with audio and also extracting features from music requires field knowledge and experience, we had to do some research beforehand and the following code (which extracts more than 25 feature from every single sample in our dataset) is based on information & codes available at following links:
https://towardsdatascience.com/extract-features-of-music-75a3f9bc265d 
and
https://medium.com/@sdoshi579/classification-of-music-into-different-genres-using-keras-82ab5339efe0